# CESI HumanForYou

## Préparation de l'environnement

## Imports des librairies


In [ ]:
# imports
import numpy as np
import os
import pandas as pd
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from numpy.random import default_rng

Affichage des graphs

In [ ]:

# stabilité du notebook d'une exécution à l'autre
random=default_rng(42) 

# jolies figures directement dans le notebook


plt.rcParams['figure.figsize'] = [15,10] 
plt.rcParams["figure.dpi"] = 250
plt.rcParams['axes.labelsize'] = 6
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8

# où sauver les figures
PROJECT_ROOT_DIR = "."
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images") # le dossier doit exister

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

import warnings
warnings.filterwarnings('ignore')

## Import des données

In [ ]:
#Import des données depuis Github de Anthony Lorendeaux
general_url = "https://raw.githubusercontent.com/anthonylorendeaux/CESI-IA-CSV/master/general_data.csv"
manager_url = "https://raw.githubusercontent.com/anthonylorendeaux/CESI-IA-CSV/master/manager_survey_data.csv"
employee_url = "https://raw.githubusercontent.com/anthonylorendeaux/CESI-IA-CSV/master/employee_survey_data.csv"
in_time_url = "https://raw.githubusercontent.com/anthonylorendeaux/CESI-IA-CSV/master/in_time.csv"
out_time_url = "https://raw.githubusercontent.com/anthonylorendeaux/CESI-IA-CSV/master/out_time.csv"

#Lecture des csv
general_info_data = pd.read_csv(general_url)
manager_survey_data = pd.read_csv(manager_url)
employee_survey_data = pd.read_csv(employee_url)
in_time_data = pd.read_csv(in_time_url)
out_time_data = pd.read_csv(out_time_url)

## Visualisation des données
Dans un premier temps, nous pouvons observer les premières lignes de notre dataset en utilisant la méthode **head()**, afin de vérifier que l'import se soit bien passé.

In [ ]:
general_info_data.head()

Nous pouvons obtenir une description du dataset avec la méthode **info()**. 
Ce qui nous permettra de repérer les attributs contenant des données vides et de connaitre le type des valeurs de chaque attribut.

La fonction **shape()** quand à elle nous permet combien de ligne et de colonne nous avons dans notre csv.

In [ ]:
print('Shape of in_time :',general_info_data.shape)
general_info_data.info()

In [ ]:
manager_survey_data.head()

In [ ]:
print('Shape of in_time :',manager_survey_data.shape)
manager_survey_data.info()

In [ ]:
employee_survey_data.head()

In [ ]:
print('Shape of in_time :',employee_survey_data.shape)
employee_survey_data.info()

In [ ]:
in_time_data.head()

In [ ]:
print('Shape of in_time :',in_time_data.shape)
in_time_data.info()

In [ ]:
out_time_data.head()

In [ ]:
print('Shape of in_time :',out_time_data.shape)
out_time_data.info()